rs72685323 is on chrom 14:51359100 C>G, we use the cross validation fold with chrom 14 in test set (split 6)

In [1]:
snp='rs72685323'
split=6
snp_chrom="chr14"
snp_pos=51359100

In [2]:
#load dragonn tutorial utilities 
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')


In [3]:
from keras.models import load_model

Using TensorFlow backend.


In [4]:
import pysam
# SNP is at position 100 in the 200 bp MPRA construct 
def get_inputs(chrom,snp_pos,left_flank=99,right_flank=100,ref="/mnt/data/annotations/by_release/hg19.GRCh37/hg19.genome.fa"):
    ref=pysam.FastaFile(ref)
    ltrdict = {'a':[1,0,0,0],'c':[0,1,0,0],'g':[0,0,1,0],'t':[0,0,0,1], 'n':[0,0,0,0],'A':[1,0,0,0],'C':[0,1,0,0],'G':[0,0,1,0],'T':[0,0,0,1],'N':[0,0,0,0]}
    snp_pos=snp_pos-1 #convert to 0-indexing
    left_flank=ref.fetch(chrom,snp_pos-left_flank,snp_pos)
    print(len(left_flank))
    right_flank=ref.fetch(chrom,snp_pos+1,snp_pos+right_flank+1)
    print(len(right_flank))
    encoded=dict() 
    seqs=dict()
    for base in ['A','C','G','T']: 
        curseq=left_flank+base+right_flank
        seqs[base]=curseq
        encoded[base]=np.expand_dims(np.expand_dims(np.array([ltrdict.get(x,[0,0,0,0]) for x in curseq]),0),0)
    return encoded,seqs


In [5]:
##load the model 
from concise.metrics import tpr, tnr, fpr, fnr, precision, f1
from kerasAC.metrics import recall, specificity, fpr, fnr, precision, f1
from kerasAC.custom_losses import get_ambig_binary_crossentropy
from kerasAC.generators import * 
from kerasAC.predict import get_model_layer_functor, get_layer_outputs
custom_objects={"sensitivity":tpr,
                "specificity":tnr,
                "fpr":fpr,
                "fnr":fnr,
                "precision":precision,
                "f1":f1,
               "ambig_binary_crossentropy":get_ambig_binary_crossentropy()}

model=load_model("/srv/scratch/annashch/deeplearning/gecco/crossvalid/v4/gecco.classification.SummitWithin200bpCenter."+str(split),custom_objects=custom_objects)

In [6]:
#get deepLIFT function 
from deeplift.conversion import kerasapi_conversion as kc
import keras.backend as K 
deeplift_model = kc.convert_model_from_saved_files("/srv/scratch/annashch/deeplearning/gecco/crossvalid/v4/gecco.classification.SummitWithin200bpCenter."+str(split),verbose=False)

In [7]:
#get the deeplift score with respect to the logit                                                                                                                    
score_func = deeplift_model.get_target_contribs_func(
find_scores_layer_idx=0,
    target_layer_idx=-2)


In [8]:
import numpy as np 
input_references = [np.array([0.3, 0.2, 0.2, 0.3])[None, None, None, :]]

 We have the fasta sequence from "CC_pilot-MPRA.fa"

>rs72685323_A_wC
TGTGTTTGATTTATTGAGGAAATGTTCTTCAGGAAAAAATCCAAAAGGGAATGAGGGTAGTAGCACTGGAAAGGGGGAAGAGCAGAACAAGGACGTGACCTCAGGCAAAGTCTAGCTTTGGCCTGATTCAGGCGGGGTGCTAAGCTTTGAAGCAAAGGGCCTGTTGTTTCTGCCCTCCGCTAAGGTAGGCAGTGGAGGAG



In [9]:
mpra_seq="TGTGTTTGATTTATTGAGGAAATGTTCTTCAGGAAAAAATCCAAAAGGGAATGAGGGTAGTAGCACTGGAAAGGGGGAAGAGCAGAACAAGGACGTGACCTCAGGCAAAGTCTAGCTTTGGCCTGATTCAGGCGGGGTGCTAAGCTTTGAAGCAAAGGGCCTGTTGTTTCTGCCCTCCGCTAAGGTAGGCAGTGGAGGAG"

In [10]:
len(mpra_seq)

200

In [11]:
mpra_seq[99] #reference allele "C" at position 100 in the sequence , 99 bp left flank, 100 bp right flank 

'C'

In [12]:
## extract the 1kb region centered at the SNP 
X,seqs=get_inputs(snp_chrom,snp_pos)


99
100


In [13]:
X['A'].shape

(1, 1, 200, 4)

In [14]:
seqs['C']

'tgtgtttgatttattgaggaaatgttcttcaggaaaaaatccaaaagggaatgagggtagtagcactggaaagggggaagagcagaacaaggacgtgacCtcaggcaaagtctagctttggcctgattcaggcggggtgctaagctttgaagcaAAGGGCCTGTTGTTTCTGCCCTCCGCTAAGGTAGGCAGTGGAGGAG'

In [15]:
#verify we have extracted the same sequence that was used in the MPRA 
seqs['C'].upper()==mpra_seq.upper()

True

In [16]:
#Now, pad to 1000 bp by using a 500 bp flank 
X,seqs=get_inputs(snp_chrom,snp_pos,left_flank=499,right_flank=500)
X['A'].shape

499
500


(1, 1, 1000, 4)

In [17]:
## pretty print
tasks=['Healthy','Tumor','SW480','HCT116','COLO205']

print('\t'+'\t'.join(tasks))
for base in ['A','C','G','T']:
    print(base+'\t'+'\t'.join([str(round(i,3)) for i in model.predict(X[base])[0]]))

	Healthy	Tumor	SW480	HCT116	COLO205
A	0.022	0.167	0.24	0.038	0.122
C	0.02	0.162	0.105	0.043	0.096
G	0.031	0.262	0.305	0.075	0.154
T	0.019	0.139	0.125	0.032	0.093


In [ ]:
split

In [ ]:
## Healthy	Tumor	SW480	HCT116	COLO205 (task order)
dl_scores_A=dict() 
for t_index in range(5):
    dl_scores_A[tasks[t_index]]=score_func(
                        task_idx=t_index,
                        input_data_list=[X['A']],
                        batch_size=1,
                        progress_update=None,
                        input_references_list=input_references)

In [ ]:
## Healthy	Tumor	SW480	HCT116	COLO205 (task order)
dl_scores_C=dict() 
for t_index in range(5):
    dl_scores_C[tasks[t_index]]=score_func(
                        task_idx=t_index,
                        input_data_list=[X['C']],
                        batch_size=1,
                        progress_update=None,
                        input_references_list=input_references)

In [ ]:
## Healthy	Tumor	SW480	HCT116	COLO205 (task order)
dl_scores_G=dict() 
for t_index in range(5):
    dl_scores_G[tasks[t_index]]=score_func(
                        task_idx=t_index,
                        input_data_list=[X['G']],
                        batch_size=1,
                        progress_update=None,
                        input_references_list=input_references)

In [ ]:
## Healthy	Tumor	SW480	HCT116	COLO205 (task order)
dl_scores_T=dict() 
for t_index in range(5):
    dl_scores_T[tasks[t_index]]=score_func(
                        task_idx=t_index,
                        input_data_list=[X['T']],
                        batch_size=1,
                        progress_update=None,
                        input_references_list=input_references)

In [ ]:
from dragonn.vis import  plot_seq_importance

## Healthy DeepLIFT Scores 

In [ ]:
ylim=(-.25,0.25)

In [ ]:
tmp1=dl_scores_A['Healthy'][0].squeeze()[400:600]
tmp2=X['A'].squeeze()[400:600]
plot_seq_importance(tmp1,tmp2,ylim=ylim)

In [ ]:
tmp3=dl_scores_C['Healthy'][0].squeeze()[400:600]
tmp4=X['C'].squeeze()[400:600]
plot_seq_importance(tmp3,tmp4,ylim=ylim)

In [ ]:
tmp5=dl_scores_G['Healthy'][0].squeeze()[400:600]
tmp6=X['G'].squeeze()[400:600]
plot_seq_importance(tmp5,tmp6,ylim=ylim)

In [ ]:
tmp7=dl_scores_T['Healthy'][0].squeeze()[400:600]
tmp8=X['T'].squeeze()[400:600]
plot_seq_importance(tmp7,tmp8,ylim=ylim)

In [ ]:
#delta track for the alternate G allele - reference C allele (with G at position 100)
plot_seq_importance(tmp5-tmp3,tmp6,ylim=ylim)

## Tumor

In [ ]:
tmp1=dl_scores_A['Tumor'][0].squeeze()[400:600]
tmp2=X['A'].squeeze()[400:600]
plot_seq_importance(tmp1,tmp2,ylim=ylim)

In [ ]:
tmp3=dl_scores_C['Tumor'][0].squeeze()[400:600]
tmp4=X['C'].squeeze()[400:600]
plot_seq_importance(tmp3,tmp4,ylim=ylim)

In [ ]:
tmp5=dl_scores_G['Tumor'][0].squeeze()[400:600]
tmp6=X['G'].squeeze()[400:600]
plot_seq_importance(tmp5,tmp6,ylim=ylim)

In [ ]:
tmp7=dl_scores_T['Tumor'][0].squeeze()[400:600]
tmp8=X['T'].squeeze()[400:600]
plot_seq_importance(tmp7,tmp8,ylim=ylim)

In [ ]:
#delta track for the alternate G allele - reference C allele (with G at position 100)
plot_seq_importance(tmp5-tmp3,tmp6,ylim=ylim)

## SW480

In [ ]:
tmp1=dl_scores_A['SW480'][0].squeeze()[400:600]
tmp2=X['A'].squeeze()[400:600]
plot_seq_importance(tmp1,tmp2,ylim=ylim)

In [ ]:
tmp3=dl_scores_C['SW480'][0].squeeze()[400:600]
tmp4=X['C'].squeeze()[400:600]
plot_seq_importance(tmp3,tmp4,ylim=ylim)

In [ ]:
tmp5=dl_scores_G['SW480'][0].squeeze()[400:600]
tmp6=X['G'].squeeze()[400:600]
plot_seq_importance(tmp5,tmp6,ylim=ylim)

In [ ]:
tmp7=dl_scores_T['SW480'][0].squeeze()[400:600]
tmp8=X['T'].squeeze()[400:600]
plot_seq_importance(tmp7,tmp8,ylim=ylim)

In [ ]:
#delta track for the alternate G allele - reference C allele (with G at position 100)
plot_seq_importance(tmp5-tmp3,tmp6,ylim=ylim)

## COLO205

In [ ]:
tmp1=dl_scores_A['COLO205'][0].squeeze()[400:600]
tmp2=X['A'].squeeze()[400:600]
plot_seq_importance(tmp1,tmp2,ylim=ylim)

In [ ]:
tmp3=dl_scores_C['COLO205'][0].squeeze()[400:600]
tmp4=X['C'].squeeze()[400:600]
plot_seq_importance(tmp3,tmp4,ylim=ylim)

In [ ]:
tmp5=dl_scores_G['COLO205'][0].squeeze()[400:600]
tmp6=X['G'].squeeze()[400:600]
plot_seq_importance(tmp5,tmp6,ylim=ylim)

In [ ]:
tmp7=dl_scores_T['COLO205'][0].squeeze()[400:600]
tmp8=X['T'].squeeze()[400:600]
plot_seq_importance(tmp7,tmp8,ylim=ylim)

In [ ]:
#delta track for the alternate G allele - reference C allele (with G at position 100)
plot_seq_importance(tmp5-tmp3,tmp6,ylim=ylim)

## HCT116

In [ ]:
tmp1=dl_scores_A['HCT116'][0].squeeze()[400:600]
tmp2=X['A'].squeeze()[400:600]
plot_seq_importance(tmp1,tmp2,ylim=ylim)

In [ ]:
tmp3=dl_scores_C['HCT116'][0].squeeze()[400:600]
tmp4=X['C'].squeeze()[400:600]
plot_seq_importance(tmp3,tmp4,ylim=ylim)

In [ ]:
tmp5=dl_scores_G['HCT116'][0].squeeze()[400:600]
tmp6=X['G'].squeeze()[400:600]
plot_seq_importance(tmp5,tmp6,ylim=ylim)

In [ ]:
tmp7=dl_scores_T['HCT116'][0].squeeze()[400:600]
tmp8=X['T'].squeeze()[400:600]
plot_seq_importance(tmp7,tmp8,ylim=ylim)

In [ ]:
#delta track for the alternate G allele - reference C allele (with G at position 100)
plot_seq_importance(tmp5-tmp3,tmp6,ylim=ylim)

In [ ]:
import pandas as pd 
mpra=pd.read_csv("/srv/scratch/annashch/gecco/20190207_Pilot_Data_Freeze_1/CC_Pilot_HCT116_SNPS_20180208.out",header=0,sep='\t')

In [ ]:
mpra[mpra['SNP']==snp][['SNP','window','strand','allele','A.log2FC','B.log2FC']]

In [ ]:
import pandas as pd 
mpra=pd.read_csv("/srv/scratch/annashch/gecco/20190207_Pilot_Data_Freeze_1/CC_Pilot_SW480_SNPS_20180208.out",header=0,sep='\t')

In [ ]:
mpra[mpra['SNP']==snp][['SNP','window','strand','allele','A.log2FC','B.log2FC']]